# Sistema de Detecção de Intrusões em Redes (IDS)
Neste projeto, irei construir um modelo de Machine Learning para identificar e classificar tráfego de rede malicioso. O projeto envolve aprendizagem supervisionada para classificação, utilizando um conjunto de dados rotulado onde o alvo é uma categoria que representa o tipo de ataque ou tráfego normal.

1. Eu utilizarei o seguinte pipeline baseado no framework **CRISP-DM**:

2. Definir o problema de negócio.

3. Coletar os dados e obter uma visão geral deles.

4. Dividir os dados em conjuntos de treino e teste.

5. Explorar os dados (análise exploratória de dados - EDA).

6. Engenharia de features, limpeza e pré-processamento de dados.

7. Treinamento do modelo, comparação, seleção de features e ajuste de hiperparâmetros.

8. Teste e avaliação final do modelo de produção.

9. Concluir e interpretar os resultados do modelo.

10. Implantar.

Neste **notebook**, irei realizar a **análise exploratória de dados (EDA**), cobrindo as etapas 1 a 4 do pipeline acima. O objetivo principal aqui é revelar **insights** que nos darão informações valiosas sobre os **padrões de tráfego de ataque** com base nas features disponíveis. Assim, mesmo antes de construir um modelo, será possível entender a **natureza do tráfego malicioso** e ajudar na **defesa da rede**. Além disso, abordarei essas etapas em detalhes a seguir, explicando a lógica por trás de cada decisão.


# 1. Problema de Negócio
Equipes de segurança cibernética frequentemente enfrentam a difícil tarefa de monitorar grandes volumes de tráfego de rede para identificar possíveis ataques. A detecção manual é inviável, e sistemas baseados em regras fixas podem falhar ao identificar novas ameaças. O problema reside em ter uma ferramenta que possa, de forma autônoma e precisa, sinalizar atividades suspeitas para que as equipes de segurança possam agir proativamente.

- #### **1.1 Qual é o Contexto?**
Em um ambiente de rede, a eficácia de um sistema de segurança é avaliada com base em Indicadores Chave de Desempenho (KPIs). Três métricas essenciais para um Sistema de Detecção de Intrusões (IDS) são:
     1. **Recall (Taxa de Detecção de Ataques):** Mede a porcentagem de ataques reais que o modelo consegue identificar corretamente. Um alto Recall é o KPI mais crítico, pois garante que o sistema está sinalizando a maioria das ameaças.
     2. **Taxa de Falsos Positivos:** Representa a proporção de tráfego normal que é incorretamente classificado como um ataque. A meta é manter essa taxa o mais baixa possível para evitar "fadiga de alerta" na equipe de segurança, que pode levar ao desprezo de alertas legítimos.
     3. **Tempo de Detecção:** O tempo que o sistema leva para identificar um ataque após ele ocorrer. Um tempo de detecção menor é crucial para uma resposta rápida e eficaz, minimizando o impacto de uma intrusão.
Esses **KPIs** ajudam as equipes de segurança a **avaliar a eficácia** das suas estratégias e a mensurar o nível de proteção da rede.

Para maximizar a segurança e a eficiência, o objetivo é maximizar o Recall, minimizando a Taxa de Falsos Positivos e o Tempo de Detecção.

- #### **Quais são os Objetivos do Projeto?**
    - 1. Identificar os padrões de tráfego associados a ataques cibernéticos.

    - 2. Construir um modelo de classificação (Random Forest) capaz de prever com alta precisão a ocorrência de um ataque.

    - 3. Oferecer uma análise detalhada dos fatores que mais contribuem para a detecção de ataques, fornecendo insights para aprimorar as regras de segurança existentes.

- #### **Quais são os benefícios?**
    - 1. **Redução de Custos:** Automatiza a detecção, reduzindo a necessidade de monitoramento manual intensivo.

    - 2. **Melhora na Postura de Segurança:** Aumenta a capacidade de detectar e responder a ameaças em tempo hábil.

    - 3. **Análise de Dados: Fornece insights** sobre os tipos de ataques mais comuns e suas características, permitindo uma melhor defesa.

    - 4. **Proteção de Ativos:** Minimiza a exposição a riscos cibernéticos, protegendo a integridade da rede e dos dados.

## Importando Bibliotecas

In [6]:
# Importação das bibliotecas para manipulação e visualização de dados.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Divisão dos dados.
from sklearn.model_selection import train_test_split

# Para ignorar avisos
import warnings
warnings.filterwarnings('ignore')

### Configurações de Visualização
# Magic Command para exibir os gráficos no notebook
%matplotlib inline

# Estilo para os gráficos
mpl.style.use('ggplot') 
mpl.rcParams['axes.facecolor']      = 'white'
mpl.rcParams['axes.linewidth']      = 1
mpl.rcParams['xtick.color']         = 'black'
mpl.rcParams['ytick.color']         = 'black'
mpl.rcParams['grid.color']          = 'lightgray'
mpl.rcParams['figure.dpi']          = 150
mpl.rcParams['axes.grid']           = True
mpl.rcParams['font.size']           = 12

# Definição e aplicação de uma paleta de cores personalizada
color_palette = ['#023047', '#e85d04', '#0077b6', '#ff8200', '#0096c7', '#ff9c33']
sns.set_palette(sns.color_palette(color_palette))
sns.color_palette(color_palette)

[(0.00784313725490196, 0.18823529411764706, 0.2784313725490196),
 (0.9098039215686274, 0.36470588235294116, 0.01568627450980392),
 (0.0, 0.4666666666666667, 0.7137254901960784),
 (1.0, 0.5098039215686274, 0.0),
 (0.0, 0.5882352941176471, 0.7803921568627451),
 (1.0, 0.611764705882353, 0.2)]

# 2. Entendendo os Dados
- O dataset UNSW-NB15 foi gerado pelo Cyber Centre da UNSW (Universidade de New South Wales) em 2015 e é amplamente utilizado em pesquisas de segurança de rede. Ele contém uma mistura de tráfego de rede normal e de diversos tipos de ataques, tornando-o ideal para o nosso problema de classificação.

- Esse dataset foi coletado do kaggle: https://www.kaggle.com/datasets/dhoogla/nfunswnb15v2

- Ele é composto por 45 colunas, sendo a variável-alvo a coluna attack_cat, que classifica cada pacote como tráfego normal ou um dos nove tipos de ataque.


In [7]:
data_path = '../artifacts/NF-UNSW-NB15-V2.parquet'
df = pd.read_parquet(data_path)

# Visualização das primeiras 5 linhas
df.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1305,21,6,1.0,9,1,193,3,24,24,...,0,7240,0,0,0,0,0,331.0,0,Benign
1,1305,21,6,1.0,261,5,469,7,24,24,...,8688,8688,18944,74,0,0,0,230.0,0,Benign
2,1305,21,6,1.0,481,9,750,11,24,24,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
3,1305,21,6,1.0,701,13,1054,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
4,1305,21,6,1.0,1031,19,1474,21,24,24,...,14480,13032,64256,251,0,0,0,230.0,0,Benign


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986745 entries, 0 to 1986744
Data columns (total 43 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   L4_SRC_PORT                  int32  
 1   L4_DST_PORT                  int32  
 2   PROTOCOL                     int16  
 3   L7_PROTO                     float32
 4   IN_BYTES                     int32  
 5   IN_PKTS                      int16  
 6   OUT_BYTES                    int32  
 7   OUT_PKTS                     int16  
 8   TCP_FLAGS                    int8   
 9   CLIENT_TCP_FLAGS             int8   
 10  SERVER_TCP_FLAGS             int8   
 11  FLOW_DURATION_MILLISECONDS   int32  
 12  DURATION_IN                  int16  
 13  DURATION_OUT                 int16  
 14  MIN_TTL                      int16  
 15  MAX_TTL                      int16  
 16  LONGEST_FLOW_PKT             int16  
 17  SHORTEST_FLOW_PKT            int16  
 18  MIN_IP_PKT_LEN               int16  
 19  

### Dicionário de Dados
A seguir, a descrição de todas as 43 features do dataset NF-UNSW-NB15-V2.parquet:
1.  **L4_SRC_PORT:** Porta de origem da camada 4 (transporte)
1.  **L4_DST_PORT:** Porta de destino da camada 4 (transporte).
1.  **PROTOCOL:** O protocolo de rede (ex: TCP, UDP, ICMP).
1.  **L7_PROTO:** Protocolo da camada 7 (aplicação).
1.  **IN_BYTES:** Número de bytes de entrada (enviados da origem para o destino).
1.  **IN_PKTS:** Número de pacotes de entrada.
1.  **OUT_BYTES:** Número de bytes de saída (enviados do destino para a origem).
1.  **OUT_PKTS:** Número de pacotes de saída.
1.  **TCP_FLAGS:** Flags TCP.
1.  **CLIENT_TCP_FLAGS:** Flags TCP do lado do cliente.
1.  **SERVER_TCP_FLAGS:** Flags TCP do lado do servidor.
1.  **FLOW_DURATION_MILLISECONDS:** Duração total do fluxo de conexão em milissegundos.
1.  **DURATION_IN:** Duração do fluxo de entrada.
1.  **DURATION_OUT:** Duração do fluxo de saída.
1.  **MIN_TTL:** Menor valor de Time-to-Live (Tempo de Vida).
1.  **MAX_TTL:** Maior valor de Time-to-Live.
1.  **LONGEST_FLOW_PKT:** Tamanho do pacote de maior duração.
1.  **SHORTEST_FLOW_PKT:** Tamanho do pacote de menor duração.
1.  **MIN_IP_PKT_LEN:** Tamanho mínimo do pacote IP.
1.  **MAX_IP_PKT_LEN:** Tamanho máximo do pacote IP.
1.  **SRC_TO_DST_SECOND_BYTES:** Taxa de transferência de bytes por segundo (origem para destino).
1.  **DST_TO_SRC_SECOND_BYTES:** Taxa de transferência de bytes por segundo (destino para origem).
1.  **RETRANSMITTED_IN_BYTES:** Bytes de entrada retransmitidos.
1.  **RETRANSMITTED_IN_PKTS:** Pacotes de entrada retransmitidos.
1.  **RETRANSMITTED_OUT_BYTES:** Bytes de saída retransmitidos.
1.  **RETRANSMITTED_OUT_PKTS:** Pacotes de saída retransmitidos.
1.  **SRC_TO_DST_AVG_THROUGHPUT:** Vazão média (throughput) da origem para o destino.
1.  **DST_TO_SRC_AVG_THROUGHPUT:** Vazão média (throughput) do destino para a origem.
1.  **NUM_PKTS_UP_TO_128_BYTES:** Número de pacotes de até 128 bytes.
1.  **NUM_PKTS_128_TO_256_BYTES:** Número de pacotes de 128 a 256 bytes.
1.  **NUM_PKTS_256_TO_512_BYTES:** Número de pacotes de 256 a 512 bytes.
1.  **NUM_PKTS_512_TO_1024_BYTES:** Número de pacotes de 512 a 1024 bytes.
1.  **NUM_PKTS_1024_TO_1514_BYTES:** Número de pacotes de 1024 a 1514 bytes.
1.  **TCP_WIN_MAX_IN:** Tamanho máximo da janela TCP de entrada.
1.  **TCP_WIN_MAX_OUT:** Tamanho máximo da janela TCP de saída.
1.  **ICMP_TYPE:** Tipo de mensagem ICMP.
1.  **ICMP_IPV4_TYPE:** Tipo de mensagem ICMP para IPv4.
1.  **DNS_QUERY_ID:** ID da consulta DNS.
1.  **DNS_QUERY_TYPE:** Tipo de consulta DNS.
1.  **DNS_TTL_ANSWER:** Time-to-Live da resposta DNS.
1.  **FTP_COMMAND_RET_CODE:** Código de retorno de comando FTP.
1.  **Label:** A variável-alvo binária (1 para ataque, 0 para normal).
1.  **Attack:** A variável-alvo que indica a categoria do ataque.

In [9]:
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns.')

The dataset has 1986745 rows and 43 columns.


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
L4_SRC_PORT,1986745.0,3.280227e+04,1.901644e+04,0.0,16248.0,32841.0,49308.0,6.553500e+04
L4_DST_PORT,1986745.0,1.364448e+04,1.958649e+04,0.0,25.0,1249.0,24196.0,6.553500e+04
PROTOCOL,1986745.0,7.760431e+00,6.024499e+00,0.0,6.0,6.0,6.0,2.550000e+02
L7_PROTO,1986745.0,3.391212e+00,1.444591e+01,0.0,0.0,0.0,1.0,2.480000e+02
IN_BYTES,1986745.0,5.439120e+03,7.698736e+04,1.0,576.0,2059.0,3728.0,3.024141e+07
IN_PKTS,1986745.0,4.183880e+01,8.641119e+01,1.0,9.0,24.0,48.0,2.289400e+04
OUT_BYTES,1986745.0,4.209057e+04,1.707662e+05,0.0,824.0,3080.0,20194.0,1.465852e+07
OUT_PKTS,1986745.0,5.331105e+01,1.273173e+02,0.0,10.0,25.0,46.0,1.107800e+04
TCP_FLAGS,1986745.0,2.214884e+01,9.451058e+00,0.0,24.0,27.0,27.0,3.100000e+01
CLIENT_TCP_FLAGS,1986745.0,2.180256e+01,9.415313e+00,0.0,24.0,27.0,27.0,3.100000e+01


## Alguns insights:

1. A maioria dos fluxos de rede tem uma duração muito curta, com a mediana em 0. No entanto, o valor máximo de **FLOW_DURATION_MILLISECONDS** é extremamente alto, indicando que uma pequena parcela das conexões é muito longa. Isso pode ser um indicador de atividades de exfiltração de dados ou ataques de longa duração que precisam de uma investigação mais aprofundada.
2. Em média, os fluxos de rede têm um baixo volume de bytes, mas os valores máximos de **IN_BYTES** e **OUT_BYTES** são na casa dos milhões. Essa assimetria extrema (valores muito altos) sugere a presença de outliers que representam a transferência de arquivos grandes, o que pode ser tanto uma atividade normal quanto um comportamento suspeito a ser analisado.
3. A grande maioria dos fluxos tem zero retransmissões de pacotes, mas os valores máximos em **RETRANSMITTED_IN_PKTS** e **RETRANSMITTED_OUT_PKTS** são anormalmente altos. Isso é um sinal ruim e pode indicar problemas de conexão, congestionamento de rede ou, mais grave, um ataque de negação de serviço **(DoS/DDoS)** que causa perda de pacotes.
4. A média da variável-alvo Label é de apenas 0.038. Isso nos mostra que apenas cerca de 3.8% do tráfego de rede é classificado como ataque. Essa é uma característica de um dataset altamente desbalanceado, o que exigirá técnicas especiais (como oversampling ou undersampling) na fase de pré-processamento para garantir que o modelo não ignore os casos de ataque.
5. Observando os valores mínimo e máximo em todas as colunas, parece que não há valores negativos ou inconsistentes que precisem de limpeza imediata, mas a presença de valores extremos (outliers) em várias colunas será uma consideração importante.

In [11]:
#Identificar valores nulos
df.isnull().sum()

L4_SRC_PORT                    0
L4_DST_PORT                    0
PROTOCOL                       0
L7_PROTO                       0
IN_BYTES                       0
IN_PKTS                        0
OUT_BYTES                      0
OUT_PKTS                       0
TCP_FLAGS                      0
CLIENT_TCP_FLAGS               0
SERVER_TCP_FLAGS               0
FLOW_DURATION_MILLISECONDS     0
DURATION_IN                    0
DURATION_OUT                   0
MIN_TTL                        0
MAX_TTL                        0
LONGEST_FLOW_PKT               0
SHORTEST_FLOW_PKT              0
MIN_IP_PKT_LEN                 0
MAX_IP_PKT_LEN                 0
SRC_TO_DST_SECOND_BYTES        0
DST_TO_SRC_SECOND_BYTES        0
RETRANSMITTED_IN_BYTES         0
RETRANSMITTED_IN_PKTS          0
RETRANSMITTED_OUT_BYTES        0
RETRANSMITTED_OUT_PKTS         0
SRC_TO_DST_AVG_THROUGHPUT      0
DST_TO_SRC_AVG_THROUGHPUT      0
NUM_PKTS_UP_TO_128_BYTES       0
NUM_PKTS_128_TO_256_BYTES      0
NUM_PKTS_2

In [12]:
#Valores duplicados
df.duplicated().sum()

np.int64(0)

In [13]:
# Verificar o número de valores únicos para cada coluna
print("Número de valores únicos por coluna:")
print(df.nunique())

Número de valores únicos por coluna:
L4_SRC_PORT                    64601
L4_DST_PORT                    64625
PROTOCOL                         255
L7_PROTO                          86
IN_BYTES                       14957
IN_PKTS                          931
OUT_BYTES                      19978
OUT_PKTS                        1251
TCP_FLAGS                         17
CLIENT_TCP_FLAGS                  16
SERVER_TCP_FLAGS                  15
FLOW_DURATION_MILLISECONDS       116
DURATION_IN                      116
DURATION_OUT                      87
MIN_TTL                           13
MAX_TTL                           15
LONGEST_FLOW_PKT                1462
SHORTEST_FLOW_PKT               1147
MIN_IP_PKT_LEN                   130
MAX_IP_PKT_LEN                  1462
SRC_TO_DST_SECOND_BYTES        16140
DST_TO_SRC_SECOND_BYTES        21422
RETRANSMITTED_IN_BYTES         10535
RETRANSMITTED_IN_PKTS            537
RETRANSMITTED_OUT_BYTES        15277
RETRANSMITTED_OUT_PKTS           692
S

Preparação dos Dados IDS
- Remoção de Colunas Inúteis: Removerei as variáveis L4_SRC_PORT, L4_DST_PORT e DNS_QUERY_ID, pois elas têm um número de valores únicos muito alto, tornando-as ineficazes para a identificação de padrões de intrusão.

- Renomeação das Colunas: Renomearei algumas colunas para um formato mais claro e consistente (snake_case) para facilitar a manipulação. Além disso, a coluna Label será renomeada para intrusion_flag para melhor interpretação.

- Tratamento de Variáveis Categóricas: As variáveis que representam categorias, como PROTOCOL, TCP_FLAGS e L7_PROTO, serão convertidas em um formato apropriado para a modelagem, utilizando One-Hot Encoding. Isso permitirá que o modelo de machine learning entenda essas informações sem pressupor uma relação numérica entre elas.

In [14]:
# Lista de colunas para remover
colunas_para_remover = ['L4_SRC_PORT', 'L4_DST_PORT', 'DNS_QUERY_ID']

# Removendo as colunas do DataFrame
df = df.drop(columns=colunas_para_remover)

print(f"Colunas removidas: {colunas_para_remover}")
print("\nPrimeiras linhas do DataFrame após a remoção:")
print(df.head())

Colunas removidas: ['L4_SRC_PORT', 'L4_DST_PORT', 'DNS_QUERY_ID']

Primeiras linhas do DataFrame após a remoção:
   PROTOCOL  L7_PROTO  IN_BYTES  IN_PKTS  OUT_BYTES  OUT_PKTS  TCP_FLAGS  \
0         6       1.0         9        1        193         3         24   
1         6       1.0       261        5        469         7         24   
2         6       1.0       481        9        750        11         24   
3         6       1.0       701       13       1054        15         24   
4         6       1.0      1031       19       1474        21         24   

   CLIENT_TCP_FLAGS  SERVER_TCP_FLAGS  FLOW_DURATION_MILLISECONDS  ...  \
0                24                16                           0  ...   
1                24                24                           0  ...   
2                24                24                           0  ...   
3                24                24                           0  ...   
4                24                24                       

In [16]:
# Renomeação de colunas
novos_nomes = {
    'L7_PROTO': 'l7_protocol',
    'Label': 'intrusion_flag',
    'Attack': 'attack_type'
}
df = df.rename(columns=novos_nomes)

colunas_padronizadas = [col.lower().replace(' ', '_') for col in df.columns]
df.columns = colunas_padronizadas

print("Nomes das colunas após a padronização:")
print(df.columns.tolist())

# Definir e converter as colunas categóricas
colunas_categoricas = [
    'protocol', 'tcp_flags', 'client_tcp_flags', 'server_tcp_flags',
    'l7_protocol', 'ftp_command_ret_code', 'dns_query_type',
    'icmp_ipv4_type', 'attack_type'
]

# Certifica que as colunas existem antes de converter
for col in colunas_categoricas:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("\nTipos de dados das colunas categóricas após a conversão para 'category':")
print(df[colunas_categoricas].dtypes)

# Aplicar o One-Hot Encoding
# Cria um novo DataFrame com as colunas categóricas convertidas em colunas numéricas (0s e 1s)
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

print("Dimensões do DataFrame após o One-Hot Encoding:", df_encoded.shape)

Nomes das colunas após a padronização:
['protocol', 'l7_protocol', 'in_bytes', 'in_pkts', 'out_bytes', 'out_pkts', 'tcp_flags', 'client_tcp_flags', 'server_tcp_flags', 'flow_duration_milliseconds', 'duration_in', 'duration_out', 'min_ttl', 'max_ttl', 'longest_flow_pkt', 'shortest_flow_pkt', 'min_ip_pkt_len', 'max_ip_pkt_len', 'src_to_dst_second_bytes', 'dst_to_src_second_bytes', 'retransmitted_in_bytes', 'retransmitted_in_pkts', 'retransmitted_out_bytes', 'retransmitted_out_pkts', 'src_to_dst_avg_throughput', 'dst_to_src_avg_throughput', 'num_pkts_up_to_128_bytes', 'num_pkts_128_to_256_bytes', 'num_pkts_256_to_512_bytes', 'num_pkts_512_to_1024_bytes', 'num_pkts_1024_to_1514_bytes', 'tcp_win_max_in', 'tcp_win_max_out', 'icmp_type', 'icmp_ipv4_type', 'dns_query_type', 'dns_ttl_answer', 'ftp_command_ret_code', 'intrusion_flag', 'attack_type']

Tipos de dados das colunas categóricas após a conversão para 'category':
protocol                category
tcp_flags               category
client_t

# 3. Dividindo os Dados em Conjuntos de Treino e Teste
- Divisão dos dados: Primeiramente, dividirei a base de dados em conjuntos de treino e teste. O conjunto de teste representará dados que o modelo nunca viu antes, garantindo uma avaliação realista e justa de sua capacidade de detecção de intrusão.
- Prevenção de data leakage: A partir de agora, toda a análise e pré-processamento (como o escalonamento) serão realizados apenas no conjunto de treino. Isso evita que informações do conjunto de teste "vazem" para o modelo, permitindo uma representação realista de como ele se comportaria em cenários reais.
- Estratificação: Usarei o parâmetro stratify=y na função de divisão. Isso é crucial para o projeto, pois a base de dados é altamente desbalanceada (com muito mais tráfego normal do que intrusões). A estratificação garante que a mesma porcentagem de intrusões seja mantida tanto no conjunto de treino quanto no de teste, o que é fundamental para treinar e avaliar o modelo corretamente.

In [17]:
# Separar as features (X) da variável alvo (y)
# 'X' contém todas as colunas exceto a variável alvo 'intrusion_flag'
# 'y' contém apenas a variável alvo 'intrusion_flag'
X = df_encoded.drop('intrusion_flag', axis=1)
y = df_encoded['intrusion_flag']

In [18]:
# Dividir os dados em conjuntos de treino e teste (70% treino, 30% teste)
# O parâmetro 'stratify=y' garante a proporção correta de intrusões em ambos os conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [19]:
# Imprimir as dimensões para verificar a divisão
print(f"Dimensões do conjunto de treino (X_train): {X_train.shape}")
print(f"Dimensões do conjunto de teste (X_test): {X_test.shape}")

print("\nProporção de intrusões no conjunto de treino:")
print(y_train.value_counts(normalize=True))

print("\nProporção de intrusões no conjunto de teste:")
print(y_test.value_counts(normalize=True))

Dimensões do conjunto de treino (X_train): (1390721, 713)
Dimensões do conjunto de teste (X_test): (596024, 713)

Proporção de intrusões no conjunto de treino:
intrusion_flag
0    0.96221
1    0.03779
Name: proportion, dtype: float64

Proporção de intrusões no conjunto de teste:
intrusion_flag
0    0.96221
1    0.03779
Name: proportion, dtype: float64


# 4. Análise Exploratória de Dados (EDA)
Agora, irei explorar o conjunto de treino para entender melhor como as features se distribuem individualmente, quais as relações entre elas, as correlações, as tendências e possíveis insights dos dados. Farei isso com foco especial nas variáveis-alvo (label e attack). Para isso, realizarei análises univariadas, bivariadas e multivariadas dos dados.

Para facilitar a visualização e o uso de parâmetros como o hue do Seaborn, irei criar um único DataFrame de treino combinando as variáveis preditoras e a variável-alvo.